In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from collections import Counter
import random
# import matplotlib.pyplot as plt
import numpy as np
# from sklearn.model_selection import train_test_split
# from tqdm.notebook import tqdm

In [2]:
# Número de GPUs disponibles. Usar 0 para modo CPU.
ngpu = 1

# Fuente de la cual queremos obtener los datos
source = 'corenlp'

In [3]:
# Decide si queremos correr en gpu o cpu
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
device

device(type='cuda', index=0)

In [4]:
ix_to_cat = ['C1', 'C2', 'E1', 'E2', 'E3', 'E4','V1']
cat_to_ix = { cat: ix for ix, cat in enumerate(ix_to_cat) }

num_cat = len(ix_to_cat) # Tamaño de salida de la red

cat_to_ix

{'C1': 0, 'C2': 1, 'E1': 2, 'E2': 3, 'E3': 4, 'E4': 5, 'V1': 6}

In [5]:
ix_to_w = []
with open(f'./../Embeddings/word_labels_{source}.txt', 'r', encoding='utf-8') as f:
    ix_to_w = f.read().splitlines()
    
w_to_ix = { w: ix for ix, w in enumerate(ix_to_w) }

tam_vocab = len(ix_to_w)

ix_to_w[:5]

[',', 'de', 'que', 'la', 'a']

In [6]:
sentences = []
with open(f'./Corpus/Test/sentences-{source}.txt', 'r', encoding='utf-8') as f:
    sentences = [s.split() for s in f.read().splitlines()]

[' '.join(s) for s in sentences[:5]]

['Informa AMLO que mañana se reunirá con maestros , padres de familia y comunidades indígenas en Guelatao , Oaxaca .',
 'Explicó que seguirá el ejemplo de los mejores presidentes de México : Benito Juárez , Francisco I.',
 'Afirmó que José Antonio Meade propone otorgar becas a estudiantes indígenas , créditos a la palabra para las mujeres y la apertura de Casas de Día para adultos mayores .',
 '" No podemos actuar con irresponsabilidad , si los contratistas , los que tienen los contratos quieren invertir y hacerse cargo , que no le cueste al erario público , a la hacienda pública , adelante , estamos nosotros dispuesto a concesionarlo " , expuso al comentar que tiene una alternativa que es construir dos pistas en el aeropuerto militar de Santa Lucía , sin cerrar el actual aeropuerto .',
 '" Es decir que tengan una concesión y que les vaya muy bien , Slim ha dicho de que es un buen negocio , pues adelante , que participe en la licitación , vamos a convocar a técnicos de inobjetable hone

In [7]:
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

ixBOS = w_to_ix[BOS]
ixEOS = w_to_ix[EOS]
ixUNK = w_to_ix[UNK]

In [8]:
def unk_w_to_ix(w):
    try:
        return w_to_ix[w]
    except KeyError:
        return ixUNK

In [9]:
X = [
    # Convierto las palabras (en minúsculas) a índices
    [ unk_w_to_ix(w.lower()) for w in sent ] 
    for sent in sentences
]

[str(x) for x in X[:5]]

['[1720, 244, 2, 172, 11, 1535, 14, 180, 0, 530, 1, 292, 7, 613, 731, 8, 2838, 0, 344, 6]',
 '[155, 2, 880, 5, 365, 1, 9, 413, 567, 1, 20, 40, 862, 486, 0, 664, 1291]',
 '[183, 2, 48, 50, 34, 921, 1767, 517, 4, 589, 731, 0, 612, 4, 3, 559, 13, 17, 114, 7, 3, 1578, 1, 939, 1, 175, 13, 371, 330, 6]',
 '[10, 12, 772, 928, 14, 6891, 0, 51, 9, 2456, 0, 9, 2, 117, 9, 549, 217, 1388, 7, 2380, 1020, 0, 2, 12, 36, 6892, 21, 3066, 250, 0, 4, 3, 1464, 271, 0, 353, 0, 182, 121, 1986, 4, 7473, 10, 0, 266, 21, 1630, 2, 47, 24, 682, 2, 18, 364, 196, 1762, 8, 5, 221, 1348, 1, 945, 1162, 0, 83, 982, 5, 627, 221, 6]',
 '[10, 18, 167, 2, 320, 24, 1996, 7, 2, 67, 503, 69, 137, 0, 1126, 62, 683, 1, 2, 18, 19, 619, 876, 0, 319, 353, 0, 2, 3414, 8, 3, 2167, 0, 39, 4, 2064, 4, 1345, 1, 5210, 598, 0, 1, 6072, 1856, 7, 2, 1856, 1968, 747, 3, 351, 13, 3, 2167, 7, 2, 11, 67, 376, 1010, 268, 28, 24, 610, 28, 24, 682, 10, 0, 35, 6]']

In [10]:
X = [np.asarray(x, dtype=np.long) for x in X]

In [11]:
def np_to_long_tensor(list_of_lists):
    return [
        torch.from_numpy(l).long().to(device)
        for l in list_of_lists
    ]

In [12]:
X_test = np_to_long_tensor(X)
N_test = len(X_test)
N_test

15

In [13]:
class PersuasionLSTM(nn.Module):
    def __init__(self, D_in, D_emb, D_lstm, D_out):
        super(PersuasionLSTM, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb)#, padding_idx=0)
        # self.dropout = nn.Dropout(p=0.2)
        self.lstm = nn.LSTM(input_size=D_emb, hidden_size=D_lstm)
        self.linear = nn.Linear(in_features=D_lstm, out_features=D_out)
        self.sig = nn.Sigmoid()
        

    def forward(self, sentence):
        T = len(sentence)
        embeddings = self.embedding(sentence).view(T, 1, -1)
        # embeddings = self.dropout(embeddings)
        lstm_out, (ht, ct) = self.lstm(embeddings)
        lstm_out = lstm_out.view(T, -1)
        preact_out = self.linear(lstm_out).view(T, -1)
        return self.sig(preact_out[-1])

In [14]:
# Dimensiones de la red neuronal
D_in = tam_vocab
D_emb = 64
D_lstm = 32
D_out = num_cat

In [15]:
persuasion = PersuasionLSTM(D_in, D_emb, D_lstm, D_out).to(device)
persuasion.eval()

PersuasionLSTM(
  (embedding): Embedding(7474, 64)
  (lstm): LSTM(64, 32)
  (linear): Linear(in_features=32, out_features=7, bias=True)
  (sig): Sigmoid()
)

In [19]:
dir_path = './modelsaves/'
filename = f'model-{source}-emb_64-lstm_32-seed_72-epochs_200-last-trained'
persuasion.load_state_dict(torch.load(dir_path+filename))

<All keys matched successfully>

In [20]:
def print_ix_sentence(sentence):
    print(' '.join(ix_to_w[ix] for ix in sentence.data))

def print_ix_cat(probs):
    print('\t'.join(ix_to_cat))
    print('\t'.join([f'{p_i.item():0.3f}' for p_i in probs.data]))

In [21]:
with torch.no_grad():
    for x in X_test:
        print_ix_sentence(x)
        pred = persuasion(x)
        print_ix_cat(pred)
        print()

informa amlo que mañana se reunirá con maestros , padres de familia y comunidades indígenas en guelatao , oaxaca .
C1	C2	E1	E2	E3	E4	V1
0.685	0.989	0.000	0.015	0.000	0.000	0.000

explicó que seguirá el ejemplo de los mejores presidentes de méxico : benito juárez , francisco i.
C1	C2	E1	E2	E3	E4	V1
0.000	0.000	0.000	0.000	0.000	0.000	0.000

afirmó que josé antonio meade propone otorgar becas a estudiantes indígenas , créditos a la palabra para las mujeres y la apertura de casas de día para adultos mayores .
C1	C2	E1	E2	E3	E4	V1
0.000	0.000	0.000	0.000	0.009	0.000	0.000

" no podemos actuar con irresponsabilidad , si los contratistas , los que tienen los contratos quieren invertir y hacerse cargo , que no le cueste al erario público , a la hacienda pública , adelante , estamos nosotros dispuesto a <UNK> " , expuso al comentar que tiene una alternativa que es construir dos pistas en el aeropuerto militar de santa lucía , sin cerrar el actual aeropuerto .
C1	C2	E1	E2	E3	E4	V1
0.000	0.906	0